In [39]:
using LinearAlgebra

In [40]:
# Example usage:
A = randn(5, 8)

5×8 Matrix{Float64}:
 -1.62564   -1.84577   -0.128699  …  -1.16942     1.26451    1.47238
 -0.208826  -0.246249  -0.47711       0.559398   -1.70784   -0.791109
  1.65281    0.857832   0.456961      0.63867     1.22422   -0.0705039
 -1.03827   -0.461344   0.966711      0.465617    0.42991    0.399561
  0.167754   0.580231   0.788079      0.0147875   0.145683   2.24373

In [41]:
function householder(x)
    """Computes the Householder transformation for input vector x
    
    Returns 
    beta: float, the multiplier for future Householder reflection
    v: vector, the householder reflector vector  
    """
    sigma = dot(x[2:end],x[2:end])
    v = copy(x)

    if sigma == 0
        beta = 0
        return beta, v
    end

    sq = sqrt(x[1]^2 + sigma)
    if x[1] > 0
        v[1] += sq
    else
        v[1] -= sq
    end

    beta = 2.0 / (v[1]^2 + sigma)

    return beta, v
end


function apply_householder_col!(B, H, cidx, m)
    """Apply Householder reflection from the left (B<- H * B)
    
    Returns (inplace) 
    B: input matrix
    H: householder vectors v  
    """
    beta, v = householder(B[cidx:m, cidx])
    B[cidx:m, cidx:end] = B[cidx:m, cidx:end] - beta * v * (v' * B[cidx:m, cidx:end])
    H[cidx:end, cidx] = v / norm(v)

    return beta
end

function apply_householder_row!(B, H, ridx, n)
    """Apply Householder reflection from the right (B<- B * H)
    
    Returns (inplace) 
    B: input matrix
    H: householder vectors v  
    """
    beta, v = householder(B[ridx, ridx+1:n])
    B[ridx:end, ridx+1:n] = B[ridx:end, ridx+1:n] - (B[ridx:end, ridx+1:n] * v) * (beta * v')
    H[ridx, ridx+1:end] = v / norm(v)

end

function update_col_sim!(U, H, cidx, m)
    """Update column unitary transform (U<- H * U)
    
    Input:
    U: Left Householder matrix (orthogonal)
    H: householder vectors v
    cidx: index of column where householder transform is applied
    m: number of columns of original matrix
    
    Returns (inplace) 
    U: Left Householder matrix (orthogonal)
    """
    v = H[cidx:end, cidx]
    if cidx > 1
        v = [zeros(cidx-1);v]
    end
    
    #U[cidx:end, cidx:end] = U[cidx:end, cidx:end] - 2 * v * (v' * U[cidx:end, cidx:end])
    #show(stdout, "text/plain", I - 2 * v * v')
    U[:,:] = U - 2 * v * (v' * U)
    
    return U
    
end

function update_row_sim!(V, H, ridx, n)
    """Update row unitary transform (V<- V * H)
    
    Input:
    V: Right Householder matrix (orthogonal)
    H: householder vectors v
    ridx: index of row where householder transform is applied
    n: number of rows of original matrix
    
    Returns (inplace) 
    V: Right Householder matrix (orthogonal) 
    """
    v = H[ridx, ridx+1:end]
    v = [zeros(ridx);v]
    
    
    #V[ridx:end, ridx+1:end] = V[ridx:end, ridx+1:end] - (V[ridx:end, ridx+1:n] * v) * (2 * v')
    #show(stdout, "text/plain", I - 2 * v * v')

    V[:,:] = V - 2 * (V * v) * v'
    
    return V
    
end


function bidiagonalize(A, return_orth = false)
    """Performs matrix bidiagonalization (for future singular value decomposition computation)
    
    Input:
    A: Rectangular m x n matrix
    
    Returns (inplace) 
    B: bidiagonal matrix with same singular values as input matrix (A = U * B * V')
    """
    m, n = size(A)
    B = copy(A)
    H = copy(A)  # v vectors in householder
    
    if return_orth
        U = zeros(m, m) + I
        V = zeros(n, n) + I
    end
    
    for k = 1:min(m,n)
        
        # column
        
        apply_householder_col!(B, H, k, m)
    
        
        if return_orth
            update_col_sim!(U, H, k, m)
        end
        
        
        # row
        if k < n
            apply_householder_row!(B, H, k, n) 
            
            if return_orth 
                update_row_sim!(V, H, k, n)
            end
        end
        
                
    end
    
    if return_orth
        V[:, end] = -V[:, end] 
        U[end, :] = -U[end, :]
        return U', B, V
    else
        return B, H
    end
end



bidiagonalize (generic function with 2 methods)

In [42]:
function givens_rotation(v)
    """ Computers the 2x2 Givens Rotation matrix for a given 2d vector v.
    
    Input:
    v: 2d vector
    
    Returns
    G:  givens rotation matrix
    """
    a, b = v[1], v[2]
    if b == 0
        c = 1
        s = 0
    else
        if abs(b) > abs(a)
            tau = -a/b
            s = 1.0/sqrt(1.0+tau*tau)
            c = s*tau
        else
            tau = -b/a
            c = 1.0/sqrt(1.0+tau*tau)
            s = c*tau
        end
    end
    return [c -s;s c]
end


function svd_golub_reinsh!(B, U, V, maxiter, eps=1e-12)
    """Computes the Singular value decomposition of a bidiagonal matrix. Based on 
    https://www.cs.utexas.edu/~inderjit/public_papers/HLA_SVD.pdf
    and
    https://link.springer.com/article/10.1007/s11075-022-01459-9
    
    Returns  
    U: Left singular vector matrix
    S: Singular values matrix
    Vt: Right singular vector matrix
    """
    
    function golub_kahan_step!(B, U, V, l, k)
        # Considering B22 -> (l:k x l:k)
        
        # Step 1: compute appropriate shift
        
        if k > 2
            a, b, d = B[k-1, k-1]^2 + B[k-2, k-1]^2, B[k-1, k-1] * B[k-1, k], B[k, k]^2 + B[k-1, k]^2
            t, r = - a - d, a * d - b^2
            s_1, s_2 = 0.5 * (-t + sqrt(t^2 -4 * r)), 0.5 * (-t - sqrt(t^2 -4 * r))
            if abs(B[k, k]^2 + B[k-1, k]^2 - s_1) < abs(B[k, k]^2 + B[k-1, k]^2 - s_2)
                s = s_1 
            else
                s = s_2 
            end
        else
           s = B[2, 2]^2 + B[1, 2]^2 
        end
        
        # Step 2: Computing Givens Rotations
        alpha, beta = B[l, l]^2-s, B[l, l] * B[l, l + 1]
        v = [alpha;beta]
        for p = l:k-1
            # Right rotation
            G = givens_rotation(v)
            B[:, p:p+1] = B[:, p:p+1] * G'
            V[:, p:p+1] = V[:, p:p+1] * G'
            
            
            # Left rotation
            alpha, beta = B[p, p], B[p + 1, p] 
            v = [alpha; beta]
            G = givens_rotation(v)
            B[p:p+1, :] = G * B[p:p+1, :]
            U[:, p:p+1] = U[:, p:p+1] * G'
            
            # Updating alpha, beta and v
            if p < k -1 
                alpha, beta = B[p, p+1], B[p, p+2]
                v = [alpha;beta]
            end
        end
    end
    
    m, n = size(B)
    maxiter_ = maxiter * log2(n * m) 
    
    for iters = 1:min(maxiter, maxiter_)
        
        # Step 1 of main loop
        # Find l and k such that matrix can be blocked as B11, B22 and B33 (diag) 
        
        k = n
        for j = n:-1:2  
            #print(abs(B[j-1, j]), " ", abs(B[j-1, j]) < eps, "\n")
            k = abs(B[j-1, j]) < eps * (abs(B[j, j]) + abs(B[j-1, j-1]))  ? j-1 : k
        end
        #print("After scan k = ", k, "\n")
        
        
        #show(stdout, "text/plain", B)
        #print("\n---------------\n")
        
        if k == 1
           # Diagonal matrix, convergence obtained
           return U, B, V
        end
        
        # Step 2 of main loop
        # Performing Golub Kahan at matrix B22
        if k == n 
            # B22 should be the full matrix 
            golub_kahan_step!(B, U, V, 1, n)
    
        else 
            # B33 (diagonal) found, have to check where B22 starts (l) and ends (k)
            l = k
            for j = k:-1:2
                l = B[j-1, j] == 0 ? j-1 : l 
            end
            
            # If l is equal to k after previous scan, then l = 1 and there is no B11 matrix
            l = (l == k) ? 1 : l 
            
            golub_kahan_step!(B, U, V, l, k)
        end
    end
end


function svd_num(A, maxiter = 80)
   """Performs singular value decompositon of a rectangular m x n matrix A.
    
    Returns 
        U: Left singular vector matrix
        S: Singular values matrix
        Vt: Right singular vector matrix
    """ 
    # Flag for transpose (case n > m)
    m, n = size(A)
    transpose_flag = false
    
    if n > m
        transpose_flag = true
    end
    
    # First main step: Find bidiagonalized matrix B
    if transpose_flag
        P_l, B, P_r = bidiagonalize(A', true);
    else
        P_l, B, P_r = bidiagonalize(A, true);
    end
    
    # Second main step: perform svd_golub_reinsh algorithm iterations
    svd_golub_reinsh!(B, P_l, P_r, maxiter)
    
    # Check if transpose
    if transpose_flag
        return P_r, B', P_l
    else
        return P_l, B, P_r
    end
    
    
    
end

svd_num (generic function with 2 methods)

In [43]:
A

5×8 Matrix{Float64}:
 -1.62564   -1.84577   -0.128699  …  -1.16942     1.26451    1.47238
 -0.208826  -0.246249  -0.47711       0.559398   -1.70784   -0.791109
  1.65281    0.857832   0.456961      0.63867     1.22422   -0.0705039
 -1.03827   -0.461344   0.966711      0.465617    0.42991    0.399561
  0.167754   0.580231   0.788079      0.0147875   0.145683   2.24373

In [44]:
P_l, B, P_r = bidiagonalize(A, true);

In [45]:
show(stdout, "text/plain", opnorm(P_l * B * P_r' - A, 1))

2.831068712794149e-15

In [46]:
U, S, V = svd_num(A);

In [47]:
show(stdout, "text/plain", U)

5×5 Matrix{Float64}:
  0.844858   0.193634  0.353692    0.348738  -0.0447691
 -0.271828   0.551856  0.694509   -0.329865   0.174388
 -0.246743  -0.629438  0.613178    0.288183  -0.289636
  0.274527  -0.114118  0.0230676  -0.707375  -0.640859
  0.275821  -0.498745  0.126648   -0.431442   0.687747

In [48]:
show(stdout, "text/plain", S)

5×8 adjoint(::Matrix{Float64}) with eltype Float64:
  3.86688       1.57643e-23  -1.58091e-17   1.52206e-17  -3.64327e-18   4.18706e-18   1.46668e-16  -2.83633e-17
 -1.214e-18     3.1392       -6.38399e-20  -1.12454e-17  -2.48458e-17  -1.028e-16     3.4652e-16    1.52504e-16
  1.97365e-19  -6.0204e-17   -1.11534       9.14506e-19  -4.76717e-17  -1.18136e-16   6.1568e-17   -1.30838e-16
 -6.88194e-18  -1.86497e-15   6.38998e-14  -1.81456      -9.80644e-19   1.56437e-16   1.09585e-17   1.41789e-16
 -4.28549e-17  -3.40407e-17  -2.55064e-17   4.1076e-16   -2.08366      -8.03295e-17  -1.21405e-16   1.95175e-16

In [49]:
show(stdout, "text/plain", V')

8×8 Matrix{Float64}:
 -0.507709  -0.432068   0.101106   -0.0141972   0.0111274  -0.301468    0.359128   0.570215
 -0.457296  -0.404559  -0.343787   -0.0935981   0.254863   -0.121129   -0.506473  -0.405117
 -0.260689   0.210707  -0.0228003   0.730146   -0.444673   -0.339918   -0.036014  -0.198584
 -0.352891   0.131847   0.429664   -0.121381   -0.343437    0.41004    -0.545685   0.273654
 -0.162408  -0.233215   0.137892   -0.362054   -0.533304    0.15516     0.424414  -0.529644
 -0.14496   -0.214542  -0.160891    0.485234    0.194865    0.756449    0.247619  -0.0108526
 -0.454764   0.645007  -0.493368   -0.272266    0.0293526   0.0900743   0.210772   0.0788301
 -0.293974   0.267238   0.630987    0.0502784   0.544051   -0.079782    0.171978  -0.330995

In [50]:
show(stdout, "text/plain", opnorm(U * S * V' - A, 1))

4.9960036108132044e-15

In [51]:
# Example usage 2:
A = randn(9, 7)

9×7 Matrix{Float64}:
 -0.0800253   1.19409     2.09825     …   1.11075     -3.18072     0.106149
 -0.485211   -0.674737    0.841982        0.356937    -0.233808   -0.78999
 -0.100065   -0.612103    0.835813        0.00412636  -1.397       0.0560684
  0.557356   -0.810498    0.818767       -0.0261235    1.65778    -0.0150909
 -0.922566    0.654493    0.00237702     -0.0149955    0.0391884  -1.20029
  0.387231    0.429389    0.883068    …   1.7368       0.86703     0.390452
  0.337484    0.0853086  -0.253825       -0.888386    -0.0187609   0.486153
 -0.595768    0.472477   -1.10004        -1.14576     -0.0805163   1.43615
 -1.5212      0.811105    0.239316        0.389064    -0.81308    -0.145473

In [52]:
U, S, V = svd_num(A);

In [53]:
show(stdout, "text/plain", U)

9×9 adjoint(::Matrix{Float64}) with eltype Float64:
 -0.849438    0.0687316   -0.282726  -0.0788428   0.100955    0.295624     0.190665   -0.175119    -0.151577
 -0.123726    0.230999     0.281015   0.331459   -0.427721   -0.220012     0.353901   -0.48296      0.390704
 -0.288072   -0.0720062   -0.177736   0.430467   -0.367341   -0.249274    -0.0538492   0.698534     0.0837325
  0.266936    0.411373    -0.122946   0.229184   -0.434781    0.619162    -0.0397277   0.0103598   -0.343212
 -0.0706154   0.00462991   0.667756  -0.0881386   0.153037    0.356638     0.44891     0.430361     0.063112
 -0.0172789   0.658308    -0.152971  -0.581849   -0.101652   -0.130288    -0.0333126   0.23631      0.346588
  0.0744313  -0.262953    -0.247514   0.0955074   0.0648098   0.513404    -0.13311    -0.00166777   0.754603
  0.134284   -0.480533    -0.228347  -0.467039   -0.5222      0.00785342   0.449375    0.00769797  -0.07769
 -0.28293    -0.184541     0.460095  -0.26931    -0.41502     0.104361    -0

In [54]:
show(stdout, "text/plain", S)

9×7 Matrix{Float64}:
 -4.79736       8.90024e-20   1.65874e-22  -1.47486e-16  -1.29325e-18   1.06416e-16   2.10668e-16
  0.0           3.57549      -1.35817e-19  -4.00944e-19   4.17121e-17  -3.89999e-16   2.4284e-16
 -6.44465e-17   1.08897e-19  -2.29989       3.02486e-15   4.001e-13     2.22125e-17  -4.59242e-17
 -1.36465e-16  -1.51201e-16  -2.99742e-21   2.1891        2.4164e-12   -4.0936e-18   -7.78533e-17
  5.23492e-17   1.80367e-16   2.7641e-16    4.80735e-17   1.17582      -1.05597e-16   1.77752e-18
  1.80583e-17  -4.7044e-17   -7.66931e-17   9.99775e-17   1.47342e-19   0.880919     -5.24348e-15
 -3.37695e-17   2.77513e-17  -4.54484e-17   4.49085e-17  -7.41444e-17  -8.4563e-20    0.723971
  1.1444e-16    2.31799e-16   3.24603e-17   4.48597e-18   7.46736e-17   1.89421e-17   1.095e-17
  3.62694e-17   8.10975e-17   2.00624e-16   6.81267e-17   9.34317e-17   1.00145e-17   4.05451e-18

In [55]:
show(stdout, "text/plain", V')

7×7 adjoint(::Matrix{Float64}) with eltype Float64:
 -0.154165    0.246838    0.44993     0.0935762   0.282415   -0.787057  -0.0699557
  0.237119   -0.133293    0.488849   -0.452641    0.560318    0.356471  -0.202787
  0.646611   -0.129035    0.137054   -0.0314668  -0.0754725  -0.182844   0.711577
  0.231283   -0.687733    0.261356    0.332882   -0.291086   -0.137636  -0.436737
  0.661366    0.395533   -0.375949    0.122238    0.144355   -0.112742  -0.465102
  0.0947763   0.524262    0.573797    0.206771   -0.480744    0.331138  -0.0583302
  0.0449807   0.0194933  -0.0302938  -0.785406   -0.513717   -0.28059   -0.192822

In [56]:
show(stdout, "text/plain", opnorm(U * S * V' - A, 1))

1.0217521273503394e-14

In [57]:
# Example usage 3:
A = randn(7, 7)
A = A + A'

7×7 Matrix{Float64}:
  1.09494    2.64617    0.512076    …   0.926954  -2.84069    3.38698
  2.64617    0.957856   2.10516        -2.66119    0.649861   0.149081
  0.512076   2.10516   -0.00822262      0.742708   0.96665   -0.593146
  0.237239  -0.657437   0.220009        1.79742    1.23833    2.686
  0.926954  -2.66119    0.742708       -2.6153    -1.07338   -1.55204
 -2.84069    0.649861   0.96665     …  -1.07338    0.701525   0.531877
  3.38698    0.149081  -0.593146       -1.55204    0.531877  -1.80795

In [58]:
U, S, V = svd_num(A);

In [59]:
show(stdout, "text/plain", U)

7×7 adjoint(::Matrix{Float64}) with eltype Float64:
  0.437236    0.718072    0.0201956  -0.322743   0.262872    0.343752   -0.036861
 -0.331692    0.471174   -0.0155626   0.511897  -0.468491    0.238903   -0.359355
  0.0588731   0.162245    0.734074   -0.285986  -0.3753     -0.448021   -0.0893556
  0.21494     0.184907    0.107991    0.440348  -0.150217   -0.0486702   0.830125
 -0.566808   -0.0609897   0.50491    -0.1149     0.281462    0.520998    0.237091
  0.107393   -0.218656   -0.18255    -0.444727  -0.684003    0.456081    0.183521
 -0.562437    0.387715   -0.400704   -0.383562  -0.0303018  -0.377119    0.287265

In [60]:
show(stdout, "text/plain", S)

7×7 Matrix{Float64}:
 -6.98311       9.67671e-20  -8.23486e-19   9.87357e-18  -3.33493e-16  -6.02239e-17   1.65284e-16
  0.0          -5.62309      -2.21543e-15  -6.84432e-21  -2.97786e-17   6.00799e-17   2.37304e-15
  2.95432e-16  -2.20585e-20   0.587839     -6.99109e-18  -1.37833e-17   3.58961e-17  -3.95913e-13
 -8.99506e-17  -1.50144e-17  -2.39459e-20  -2.53113       1.68764e-17   1.05361e-16   8.31031e-14
  4.04948e-16  -1.08289e-17  -5.64992e-17   1.01729e-17  -3.50594       2.62626e-16  -9.5232e-13
  2.05297e-16   8.39369e-17  -5.22094e-17   3.61081e-16   1.761e-18     3.84678      -1.40768e-12
  7.43776e-18  -4.5759e-18   -4.4591e-17    7.98498e-17  -6.9577e-17   -4.38091e-17   4.57617

In [61]:
show(stdout, "text/plain", V')

7×7 adjoint(::Matrix{Float64}) with eltype Float64:
  0.437236   -0.331692    0.0588731   0.21494    -0.566808    0.107393  -0.562437
 -0.718072   -0.471174   -0.162245   -0.184907    0.0609897   0.218656  -0.387715
  0.0201956  -0.0155626   0.734074    0.107991    0.50491    -0.18255   -0.400704
 -0.322743    0.511897   -0.285986    0.440348   -0.1149     -0.444727  -0.383562
 -0.262872    0.468491    0.3753      0.150217   -0.281462    0.684003   0.0303018
 -0.343752   -0.238903    0.448021    0.0486702  -0.520998   -0.456081   0.377119
 -0.036861   -0.359355   -0.0893556   0.830125    0.237091    0.183521   0.287265